In [1]:
import torch, torchvision
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda
import matplotlib.pyplot as plt

import numpy as np
import matplotlib.pylab as plt
from sklearn.preprocessing import OneHotEncoder


In [2]:
train_dataset = torchvision.datasets.MNIST('./data', train=True, download=True, transform=ToTensor())
test_dataset = torchvision.datasets.MNIST('./data', train=False, download=True, transform=ToTensor())

In [64]:
def fetch(fp):
    with open(fp, "rb") as f:
        data = f.read()
    return np.frombuffer(data, dtype=np.uint8).copy()

In [68]:
X_train = fetch("./mnist/train-images-idx3-ubyte")[0x10:].reshape((-1, 28 * 28))
Y_train = fetch("./mnist/train-labels-idx1-ubyte")[8:]
X_test = fetch("./mnist/t10k-images-idx3-ubyte")[0x10:].reshape((-1, 28 * 28))
Y_test = fetch("./mnist/t10k-labels-idx1-ubyte")[8:]

In [69]:
def normalize_oneHotEncoding(X_train, Y_train, X_test, Y_test):
    
    oneHotEncoder = OneHotEncoder(sparse=False, categories='auto')
    Y_train = oneHotEncoder.fit_transform(Y_train.reshape(len(Y_train), -1))
    Y_test = oneHotEncoder.transform(Y_test.reshape(len(Y_test), -1))
    
    # Normalize data by diving by 255. All values are in range 0-255
    X_train = X_train / 255.
    X_test = X_test / 255.

    return X_train, Y_train, X_test, Y_test


In [71]:
X_train, Y_train, X_test, Y_test = normalize_oneHotEncoding(X_train, Y_train, X_test, Y_test)

/Users/MCN/opt/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [72]:
def init_weights():
    W1 = np.random.randn(300, 784) / np.sqrt(784)
    W2 = np.random.randn(10, 300) / np.sqrt(300)
    return W1, W2

In [73]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def softmax(x):
    val = np.exp(x - np.max(x))
    return val / val.sum(axis=0)

def de_sigmoid(x):
    return sigmoid(x) * (1 - sigmoid(x))

def forward(X, w1, w2):
    z1 = w1.dot(X.T)
    i1 = sigmoid(z1)
    z2 = w2.dot(i1)
    i2 = softmax(z2)
    return i1, w1, z1, i2, w2, z2



def backward(X, Y, i1, w1, z1, i2, w2, z2, n=60000):
    dz2 = i2 - Y.T
    dw2 = dz2.dot(i1.T) / n
    dz1 = w2.T.dot(dz2) * de_sigmoid(z1)
    dw1 = dz1.dot(X) / n
    return dw1, dw2

def predict(w1, w2, X, Y):
    i1, w1, z1, i2, w2, z2 = forward(X, w1, w2)
    y_hat = np.argmax(i2, axis=0)
    Y = np.argmax(Y, axis = 1)
    accuracy = (y_hat == Y).mean()
    return accuracy * 100



In [74]:
costs = []
accus = []


def train_model(X, Y, alpha=0.1, epochs=1000):
    w1, w2 = init_weights()
    for epoch in range(epochs):
        i1, w1, z1, i2, w2, z2 = forward(X, w1, w2)
        cost = -np.mean(Y*np.log(i2.T))
        dw1, dw2 = backward(X, Y, i1, w1, z1, i2, w2, z2)
        w1 = w1 - alpha * dw1
        w2 = w2 - alpha * dw2
        acc = predict(w1, w2, X, Y)
        if epoch % 100 == 0:
            print("Cost: ", cost, "Train Accuracy:", acc)
        if epoch % 10 == 0:
            costs.append(cost)
            accus.append(acc)
    return w1, w2

In [76]:
train_model(X_train, Y_train)

Cost:  0.24037718898628085 Train Accuracy: 11.375
Cost:  0.14360326597747294 Train Accuracy: 74.59166666666667


In [3]:
train_dataloader = DataLoader(train_dataset, batch_size=32)
test_dataloader = DataLoader(test_dataset, batch_size=32)

In [4]:
# GPU
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print('GPU State:', device)


GPU State: cpu


In [5]:
train_dataset[0][0].squeeze().shape

torch.Size([28, 28])

In [7]:
class FNN(nn.Module):
    def __init__(self, d1, d2):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_layers = nn.Sequential(
            nn.Linear(28*28, d1),
            nn.Sigmoid(),
            nn.Linear(d1, d2),
            nn.Softmax(),
            nn.Linear(d2, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_layers(x)
        return logits

In [9]:
model = FNN(300, 200)

In [15]:
lr = 1e-3
bs = 64
epochs = 100
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=lr)

In [16]:
for epoch in range(epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_dataloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 1000 == 999:    # print every 100 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 100:.3f}')
            running_loss = 0.0

print('Finished Training')


[1,  1000] loss: 23.009
[2,  1000] loss: 23.009
[3,  1000] loss: 23.009
[4,  1000] loss: 23.009
[5,  1000] loss: 23.009
[6,  1000] loss: 23.009
[7,  1000] loss: 23.009
[8,  1000] loss: 23.009
[9,  1000] loss: 23.009
[10,  1000] loss: 23.009
[11,  1000] loss: 23.009
[12,  1000] loss: 23.009
[13,  1000] loss: 23.009
[14,  1000] loss: 23.009
[15,  1000] loss: 23.009
[16,  1000] loss: 23.009
[17,  1000] loss: 23.009
[18,  1000] loss: 23.009
[19,  1000] loss: 23.009
[20,  1000] loss: 23.009
[21,  1000] loss: 23.009
[22,  1000] loss: 23.009
[23,  1000] loss: 23.009
[24,  1000] loss: 23.009
[25,  1000] loss: 23.009
[26,  1000] loss: 23.009
[27,  1000] loss: 23.009
[28,  1000] loss: 23.009
[29,  1000] loss: 23.009
[30,  1000] loss: 23.009
[31,  1000] loss: 23.009
[32,  1000] loss: 23.009
[33,  1000] loss: 23.009
[34,  1000] loss: 23.009
[35,  1000] loss: 23.009
[36,  1000] loss: 23.009
[37,  1000] loss: 23.009
[38,  1000] loss: 23.009
[39,  1000] loss: 23.009
[40,  1000] loss: 23.009
[41,  100

In [17]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in test_dataloader:
        images, labels = data
        # calculate outputs by running images through the network
        outputs = model(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the {len(images)} test images: {100 * correct // total} %')

Accuracy of the network on the 16 test images: 11 %


In [18]:
classes = list(range(10))
# prepare to count predictions for each class
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

# again no gradients needed
with torch.no_grad():
    for data in test_dataloader:
        images, labels = data
        outputs = model(images)
        _, predictions = torch.max(outputs, 1)
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1


# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy for class: {classname:5f} is {accuracy:.1f} %')

Accuracy for class: 0.000000 is 0.0 %
Accuracy for class: 1.000000 is 100.0 %
Accuracy for class: 2.000000 is 0.0 %
Accuracy for class: 3.000000 is 0.0 %
Accuracy for class: 4.000000 is 0.0 %
Accuracy for class: 5.000000 is 0.0 %
Accuracy for class: 6.000000 is 0.0 %
Accuracy for class: 7.000000 is 0.0 %
Accuracy for class: 8.000000 is 0.0 %
Accuracy for class: 9.000000 is 0.0 %
